In [153]:
import os
import re
import urllib.request
from bs4 import BeautifulSoup
import os
import csv
import time
import requests
from urllib.request import Request, urlopen
import pandas as pd

In [154]:
topic_Name = ["Data Science","machine Learning","OpenCV","ComputerVision","GAN","variational encoder","awsome","javascript","c++"]
number_of_Repositories  = 2
repository_list = {}
for name in topic_Name:
    repository_list[name] = []
Data = {}
col_name = ["topic","name","user","star","fork","watch","topic_tag","discription_text","discription_url","commits","branches","packages","releases","contributers","License","url"]
for name in col_name:
    Data[name] = []

In [155]:
for topic in topic_Name:
    page_number = 1
    repo_number = 0
    while len(repository_list[topic]) < number_of_Repositories:
        url = "https://github.com/search?o=desc&p="+ str(page_number) +"&q="+str(topic)+"&s=stars&type=Repositories"
        data_page = requests.get(url)
        time.sleep(0.1)
        main_data = BeautifulSoup(data_page.content,'html.parser')
        for sub_data in main_data.findAll('li',attrs={'class':'repo-list-item'}):
            repository_path = str(sub_data.findAll('a',attrs={'class':"v-align-middle"})[0].text)
            repository_user_name , repository_name = repository_path.split('/')
            repository_URL = "https://github.com/" + repository_path
            repository_page = requests.get(repository_URL)
            time.sleep(0.1)
            repository_data = BeautifulSoup(repository_page.content,'html.parser')
            # ropository data

            ## Social Count
            social_count = repository_data.findAll('a',attrs={'class':"social-count"})
            repository_watch = social_count[0].text.strip()
            repository_star = social_count[1].text.strip()
            repository_fork = social_count[2].text.strip()
            # print(repository_watch,repository_star,repository_fork)
            
            # Repository tag
            repository_topic_tag = [str(i.text)[3:-1] for i in repository_data.findAll('a',attrs={'class':"topic-tag"})]
            repository_discription = repository_data.find('div',attrs={'class':"f4"})

            # Repository Discription
            repository_discription_text = repository_discription.find('span',attrs={'class':"mr-2"}).text.strip()
            repository_discription_url = repository_discription.find('a').text if repository_discription.find('a') != None else None
            
            # Repository overall-summary
            repository_overall_summary = repository_data.find('ul',attrs={'class':'numbers-summary'})
            repository_num_summary = repository_overall_summary.findAll('span',attrs={'class':'num'})
            repository_commits = repository_num_summary[0].text.strip() if repository_num_summary[0] != None else 0
            repository_branches = repository_num_summary[1].text.strip() if repository_num_summary[1] != None else 0
            repository_packages = repository_num_summary[2].text.strip() if repository_num_summary[2] != None else 0
            repository_releases = repository_num_summary[2].text.strip() if repository_num_summary[3] != None else 0
            repository_contributers = repository_num_summary[3].text.strip() if repository_num_summary[3] != None else 0
            repository_License = repository_data.find('ul',attrs={'class':'numbers-summary'}).findAll('li')[-1].text.strip()
            repo_number += 1

            # print(repo_number,repository_path,repository_URL,repository_name,repository_star,repository_fork,repository_watch,repository_topic_tag,repository_discription_text,repository_discription_url,repository_commits,repository_branches,repository_packages,repository_releases,repository_contributers,repository_License)
            
            for a,b in zip(["topic","name","user","star","fork","watch","topic_tag","discription_text","discription_url","commits","branches","packages","releases","contributers","License","url"],[topic,repository_name,repository_user_name,repository_star,repository_fork,repository_watch,repository_topic_tag,repository_discription_text,repository_discription_url,repository_commits,repository_branches,repository_packages,repository_releases,repository_contributers,repository_License,repository_URL]):
                Data[a].append(b)
                
            repository_list[topic].append(repository_path)
            
            if (repo_number >= number_of_Repositories):
                break
        page_number += 1

In [156]:
Data_f = pd.DataFrame(Data)

In [157]:
Data_f

,topic,name,user,star,fork,watch,topic_tag,discription_text,discription_url,commits,branches,packages,releases,contributers,License,url
0,Data Science,PythonDataScienceHandbook,jakevdp,23.1k,9.9k,1.5k,"[scikit-learn, numpy, python, jupyter-notebook...",Python Data Science Handbook: full text in Jup...,http://jakevdp.github.io/PythonDataSc…,234,5,0,0,0,View license,https://github.com/jakevdp/PythonDataScienceHa...
1,Data Science,Data-Science--Cheat-Sheet,abhat222,18.4k,8.2k,1.5k,[],Cheat Sheets,None,363,3,0,0,0,8\n \n contributors,https://github.com/abhat222/Data-Science--Chea...
2,machine Learning,tensorflow,tensorflow,144k,80.9k,8.4k,"[tensorflow, machine-learning, python, deep-le...",An Open Source Machine Learning Framework for ...,https://tensorflow.org,"84,314",50,0,0,107,Apache-2.0,https://github.com/tensorflow/tensorflow
3,machine Learning,awesome-machine-learning,josephmisiti,44.3k,11.2k,3.4k,[],A curated list of awesome Machine Learning fra...,None,"1,390",2,0,0,0,View license,https://github.com/josephmisiti/awesome-machin...
4,OpenCV,opencv,opencv,43.9k,34.9k,2.6k,"[opencv, c-plus-plus, computer-vision, deep-le...",Open Source Computer Vision Library,https://opencv.org,"28,487",3,0,0,93,View license,https://github.com/opencv/opencv
5,OpenCV,openpose,CMU-Perceptual-Computing-Lab,16.7k,5k,872,"[openpose, computer-vision, machine-learning, ...",OpenPose: Real-time multi-person keypoint dete...,None,609,3,0,0,13,View license,https://github.com/CMU-Perceptual-Computing-La...
6,ComputerVision,learnopencv,spmallick,10.2k,6.7k,842,"[computer-vision, machine-learning, ai, deep-l...",Learn OpenCV : C++ and Python Examples,https://www.learnopencv.com/,857,2,0,0,0,37\n \n contributors,https://github.com/spmallick/learnopencv
7,ComputerVision,vrn,AaronJackson,4k,679,213,"[computervision, deeplearning, 3d, reconstruct...","👨 Code for ""Large Pose 3D Face Reconstruction ...",http://aaronsplace.co.uk/papers/jacks…,56,1,0,0,0,MIT,https://github.com/AaronJackson/vrn
8,GAN,pytorch-CycleGAN-and-pix2pix,junyanz,11.6k,3.4k,311,"[pytorch, gan, cyclegan, pix2pix, deep-learnin...",Image-to-Image Translation in PyTorch,None,445,3,0,0,0,View license,https://github.com/junyanz/pytorch-CycleGAN-an...
9,GAN,CycleGAN,junyanz,9.6k,1.6k,401,"[gan, generative-adversarial-network, deep-lea...",Software that can generate photos from paintin...,None,95,1,0,0,0,View license,https://github.com/junyanz/CycleGAN
